<a href="https://colab.research.google.com/github/mariatomy9/tomato-disease-predictor/blob/main/Tomato_disease_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tomato-disease prediction

In [ ]:
# importing essential libraries

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, Sequential
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
from glob import glob

In [ ]:
image_size = [224, 224]

train_path = '/content/drive/MyDrive/Leaf_disease_detection/dataset2/PlantVillage/PlantVillage/train_sample'
valid_path = '/content/drive/MyDrive/Leaf_disease_detection/dataset2/PlantVillage/PlantVillage/test_sample'

In [ ]:
resnet = ResNet50(input_shape = image_size + [3], weights = 'imagenet', include_top=False)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
# not training existing weights
for layer in resnet.layers:
  layer.trainable = False

In [ ]:
# useful in getting total number of classes
folders = glob("/content/drive/MyDrive/Leaf_disease_detection/dataset2/PlantVillage/PlantVillage/train_sample/*")
folders

['/content/drive/MyDrive/Leaf_disease_detection/dataset2/PlantVillage/PlantVillage/train_sample/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/content/drive/MyDrive/Leaf_disease_detection/dataset2/PlantVillage/PlantVillage/train_sample/Tomato___healthy']

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
# adding prediction layer
prediction = Dense(len(folders), activation='softmax')(x)

# creating model 
model = Model(inputs = resnet.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer='adam', metrics=['accuracy']
)

In [ ]:
# data augmentation
train_datagen = ImageDataGenerator(rescale= 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Leaf_disease_detection/dataset2/PlantVillage/PlantVillage/train_sample',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5559 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Leaf_disease_detection/dataset2/PlantVillage/PlantVillage/test_sample',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1389 images belonging to 2 classes.


In [ ]:
# fitting the model 
fit_model = model.fit_generator(
    training_set, 
    validation_data = test_set,
    epochs = 5,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

In [ ]:
model.save("/content/drive/MyDrive/Leaf_disease_detection/dataset2/PlantVillage/PlantVillage/model_resnet_50.h5")